<a href="https://colab.research.google.com/github/MaggieLieu/MLiS_examples/blob/master/Graph_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graphical models

# Graph CNN